In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from google.cloud.bigquery import Client

In [ ]:
client = Client()

query = """select 
*
from `aa-ai-specialisation.chicago_taxi_trips.local_taxi_trips`
where trip_start_timestamp >= '2023-01-01'
"""
job = client.query(query)
df = job.to_dataframe()
df = df.drop(columns=["pickup_census_tract", "dropoff_census_tract"], axis=1)

companies = df["company"].unique()
p_areas = df["pickup_community_area"].unique()
d_areas = df["dropoff_community_area"].unique()


preprocess_bq_query = f"""
CREATE OR REPLACE TABLE chicago_taxi_trips.processed_local_taxi_trips AS
SELECT 
    fare,
    -- Standardize numeric columns
    (trip_miles - (SELECT AVG(trip_miles) FROM chicago_taxi_trips.local_taxi_trips WHERE trip_start_timestamp >= '2023-01-01')) / 
    (SELECT STDDEV(trip_miles) FROM chicago_taxi_trips.local_taxi_trips WHERE trip_start_timestamp >= '2023-01-01') AS standardized_trip_miles,
    (trip_seconds - (SELECT AVG(trip_seconds) FROM chicago_taxi_trips.local_taxi_trips WHERE trip_start_timestamp >= '2023-01-01')) / 
    (SELECT STDDEV(trip_seconds) FROM chicago_taxi_trips.local_taxi_trips WHERE trip_start_timestamp >= '2023-01-01') AS standardized_trip_seconds,
    -- Extract month and hour from timestamp
    EXTRACT(MONTH FROM TIMESTAMP(trip_start_timestamp)) AS trip_start_month,
    EXTRACT(HOUR FROM TIMESTAMP(trip_start_timestamp)) AS trip_start_hour

"""

for c in companies:
    case_statement = f", CASE WHEN company = '{c}' THEN 1 ELSE 0 END AS company_{c}"
    preprocess_bq_query += case_statement

for p in p_areas:
    if type(p) != np.int64:
        continue
    case_statement = f", CASE WHEN pickup_community_area = {p} THEN 1 ELSE 0 END AS pickup_community_area_{p}"
    preprocess_bq_query += case_statement

for d in d_areas:
    if type(d) != np.int64:
        continue
    case_statement = f", CASE WHEN dropoff_community_area = {d} THEN 1 ELSE 0 END AS dropoff_community_area_{d}"
    preprocess_bq_query += case_statement

preprocess_bq_query_2 = f"""
    FROM 
    `aa-ai-specialisation.chicago_taxi_trips.local_taxi_trips`
WHERE
    trip_start_timestamp >= '2023-01-01' AND
    pickup_community_area IS NOT NULL AND
    dropoff_community_area IS NOT NULL AND
    company IS NOT NULL AND
    trip_miles IS NOT NULL AND
    trip_seconds IS NOT NULL AND
    fare IS NOT NULL
"""

preprocess_bq_query += preprocess_bq_query_2


query_job = client.query(preprocess_bq_query)

In [82]:
# # Wait for the query to finish
query_job.result()

# Check for errors
if query_job.errors:
    print("Query execution errors: ", query_job.errors)
else:
    print("Data preprocessed successfully.")

Data preprocessed successfully.
